## Today's Data: Wind Turbine Power

This dataset is freely available and was taken from Kaggle: https://www.kaggle.com/berkerisen/wind-turbine-scada-dataset/data

It is available as a csv file, and in this notebook we'll convert the data to a variety of formats, discuss pros/cons of each format, and review some Python packages that are useful for working with these formats.

## Text Formats

Text files are binary files that are encoded as text.  There are lots of possible **encodings** out there (full list of Python-supported encodings can be found [in the Python docs](https://docs.python.org/3/library/codecs.html)), and you generally need to know which encoding was used in order to read the file.  Lots of text editors and file readers can **infer** (i.e. guess plus trial-and-error) the encoding, though, so if a commonly-used one (like utf8) is chosen, most programs can read it fine.

When a file is easy to open with a text editor, it is said to be "**human-readable**".  Sometimes files are not easily opened, though, if they are too big, or if the data is too complex or wide.  Plus, if the data is arranged in a non-standard way, text data can be quite difficult to interpret.

In this section, we'll focus on making data **"machine-readable"**; that is, stored in a structured manner so that our programs can read them into their own data structures.

You can open any file in Python with the **open()** function.  The function also takes a **mode**, which you choose depending on the file format and what you want to do:

| mode | format | read/write | create new file? |
| :--: | :--:   | :--:       | :--:             |
| 'r'    | text   | read       | no |
| 'w'    | text   | write | yes |
| 'a'    | text   | write | no |
| 'rb'   | binary | read | no |
| 'wb'    | binary   | write | yes |
| 'ab'    | binary   | write | no |

In [1]:
with open(file='T1.csv', mode='r') as f:
    text = f.read()
text[:300]

'ï»¿Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)\n01 01 2018 00:00,380.047790527343,5.31133604049682,416.328907824861,259.994903564453\n01 01 2018 00:10,453.76919555664,5.67216682434082,519.917511061494,268.64111328125\n01 01 2018 00:20,306.3765869140'

Notice that first part?  That section is called the BOM. It helps signal things like endianness and the encoding for encoding inference, but we don't need it.  Let's read it without the BOM by specifying a different encoding:

In [2]:
with open(file='T1.csv', mode='r', encoding='utf-8-sig') as f:
    text = f.read()
text[:300]

'Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)\n01 01 2018 00:00,380.047790527343,5.31133604049682,416.328907824861,259.994903564453\n01 01 2018 00:10,453.76919555664,5.67216682434082,519.917511061494,268.64111328125\n01 01 2018 00:20,306.376586914062,'

All gone!

In the notebook, it is often useful to explicitly **print** text data, as the whitespace characters get properly handled that way:

In [3]:
print(text[:300])

Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
01 01 2018 00:00,380.047790527343,5.31133604049682,416.328907824861,259.994903564453
01 01 2018 00:10,453.76919555664,5.67216682434082,519.917511061494,268.64111328125
01 01 2018 00:20,306.376586914062,


### CSV (Comma-Seperated Values)

The turbine data is an example of *Tabular* data, which is an ordered list of same-field records.  Each field may have a different data type, but the key is that each record contains the same data types.  Tabular data can have labeled fields or not.

CSV files are a way to encode these records in text format.  Each line of the file represents one record, and fields are seperated (a.k.a. "delimited") by some character, usually commas or tabs.  If the fields are labelled, the first line of the file (called the "header") will contain the field names.

Example CSV File:
```
month,height,weight
Jan,1.2,76
Feb,1.21,77
March,1.21,76
```

CSV files are currently the most popular way of storing data, but they have so many variants that passing them around can be quite difficult.  These variants come from the format's simplicity; people keep writing their own CSV writers and parsers, which are often incompatible with other parsers.  There are plenty of options out there already, no need to write your own!

#### The csv module

Python comes with a nifty csv reader that works quite well!

In [4]:
import csv
with open('T1.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    header = next(reader)
    data = list(reader)
header

['Timestamp',
 'LV ActivePower (kW)',
 'Wind Speed (m/s)',
 'Theoretical_Power_Curve (KWh)',
 'Wind Direction (deg)']

In [5]:
data[:2]

[['01 01 2018 00:00',
  '380.047790527343',
  '5.31133604049682',
  '416.328907824861',
  '259.994903564453'],
 ['01 01 2018 00:10',
  '453.76919555664',
  '5.67216682434082',
  '519.917511061494',
  '268.64111328125']]

Once you have read the data, it can go to a Dataframe or wherever you need it.

In [6]:
import pandas as pd
df = pd.DataFrame(data=data, columns=header)
df.head()

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,01 01 2018 00:00,380.047790527343,5.31133604049682,416.328907824861,259.994903564453
1,01 01 2018 00:10,453.76919555664,5.67216682434082,519.917511061494,268.64111328125
2,01 01 2018 00:20,306.376586914062,5.21603679656982,390.900015810951,272.564788818359
3,01 01 2018 00:30,419.645904541015,5.65967416763305,516.127568975674,271.258087158203
4,01 01 2018 00:40,380.650695800781,5.57794094085693,491.702971953588,265.674285888671


In [7]:
df.dtypes

Timestamp                        object
LV ActivePower (kW)              object
Wind Speed (m/s)                 object
Theoretical_Power_Curve (KWh)    object
Wind Direction (deg)             object
dtype: object

#### Pandas.read_csv()

Pandas has a lot of different **read_()** functions, all of which can get the data into a DataFrame.  Most of this notebook will be looking at them.

The major advantage of the [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function is that it has lots of options and does good file format and data format inference.  Notice how it automatically handles the encoding here without us asking!

In [8]:
import pandas as pd
df = pd.read_csv('T1.csv')
df.head()

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [9]:
df.dtypes

Timestamp                         object
LV ActivePower (kW)              float64
Wind Speed (m/s)                 float64
Theoretical_Power_Curve (KWh)    float64
Wind Direction (deg)             float64
dtype: object

Now that's pretty simple!

### JSON: Javascript Object Notation

The next one to look at is JSON.  It's a popular format for transferring data over the web via APIs, and is also a text format. 

JSON is very similar to the text representation of a Python dictionary and lists. Let's make some JSON and you'll see what I mean:

In [10]:
data = """
{
"day": "Saturday",
"week": 3,
"isSunny": true,
"goals": ["eat breakfast", "write a book", "eat lunch"]
}
"""
print(data)


{
"day": "Saturday",
"week": 3,
"isSunny": true,
"goals": ["eat breakfast", "write a book", "eat lunch"]
}



#### The json module

Below, I'll use the built-in [**json**](https://docs.python.org/3/library/json.html) module to read the data. 

The json module has 4 main functions:

| function | read/write | file/string |
| :---:    | :----:     |  :-----:    |
| load()   |  read      | file        |
| dump()   |  write     | file        |
| loads()  |  read      | string      |
| dumps()  |  write     | string      |

In [11]:
import json
json.loads(data)

{'day': 'Saturday',
 'week': 3,
 'isSunny': True,
 'goals': ['eat breakfast', 'write a book', 'eat lunch']}

The main downside with **hand-writing** JSON is that it is very picky about getting everything right. Even though it's very readable, it should not be considered human writable.

Tabular data can also be stored in JSON in a variety of ways, called "orientations".  Pandas provides support for most of them:

- **'split'** : dict like {'index' -> [index], 'columns' -> [columns], 'data' -> [values]}
- **'records'** : list like [{column -> value}, ... , {column -> value}]
- **'index'** : dict like {index -> {column -> value}}
- **'columns'** : dict like {column -> {index -> value}}
- **'values'** : just the values array
- **'table'** : dict like {'schema': {schema}, 'data': {data}}

In [12]:
print(df.head(2).to_json(orient='records', indent=2))

[
  {
    "Timestamp":"01 01 2018 00:00",
    "LV ActivePower (kW)":380.0477905273,
    "Wind Speed (m\/s)":5.3113360405,
    "Theoretical_Power_Curve (KWh)":416.3289078249,
    "Wind Direction (deg)":259.9949035645
  },
  {
    "Timestamp":"01 01 2018 00:10",
    "LV ActivePower (kW)":453.7691955566,
    "Wind Speed (m\/s)":5.6721668243,
    "Theoretical_Power_Curve (KWh)":519.9175110615,
    "Wind Direction (deg)":268.6411132812
  }
]


## Binary Files

"Binary" files can represent any file type; conventionally, though, when someone says they have a binary file format, they mean any or all of three things:
    
1. The file doesn't open in a text editor, or shows a bunch of nonsense characters when opened in a text editor.
2. The file is meant to be read by a specific program, and/or
3. They don't know what program to use read from the file.
   
Without documentation, software availability, and version management, these can all be significant hurdles to overcome.  To illustrate it, below is a "binarized" version of the dataframe:

In [13]:
df.head(3)

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789


In [14]:
import dataframe_to_binary as df2bin
df['Timestamp'] = pd.to_numeric(pd.to_datetime(df['Timestamp'], format="%d %m %Y %H:%M"))
fmt = df2bin.to_binary_file(df, filename="hi.bin")
fmt

BinaryFormatter(header='9s19s16s29s20s', body='qdddd')

In [15]:
with open('hi.bin', 'rb') as f:
    data = f.read()
data[:500]

b'TimestampLV ActivePower (kW)Wind Speed (m/s)Theoretical_Power_Curve (KWh)Wind Direction (deg)\x00\x00D^\x8f\x86\x05\x15\xf3\xff\xff\xbf\xc3\xc0w@\xf9\xff\xff\xdf\xce>\x15@\xd3\xf2\xd94C\x05z@\xfe\xff\xff\x1f\xeb?p@\x00p\r\x11\x1b\x87\x05\x15\xf5\xff\xff\x9fN\\|@\x00\x00\x00\x80L\xb0\x16@\xae\x16\n\x10W?\x80@\x00\x00\x00\x00B\xcap@\x00\xe0\xd6\xc3\xa6\x87\x05\x15\xf7\xff\xff\x7f\x06&s@\xfb\xff\xff\xbf8\xdd\x14@\xae\x9e\xfavfnx@\xf9\xff\xff_\t\tq@\x00P\xa0v2\x88\x05\x15\xf5\xff\xff\x9fU:z@\xf8\xff\xff\x9f\x81\xa3\x16@\x08\x14\xe2B\x05!\x80@\xfe\xff\xff\x1f!\xf4p@\x00\xc0i)\xbe\x88\x05\x15\xfc\xff\xff?i\xcaw@\xfc\xff\xff\xbf\xcfO\x16@\xa7\xea\x84_?\xbb~@\xf1\xff\xff\xdf\xc9\x9ap@\x0003\xdcI\x89\x05\x15\xf5\xff\xff\x9fE&y@\xf7\xff\xff\x9f\x8cj\x16@\x07 \xddn\xfb6\x7f@\x00\x00\x00\x00B\x89p@\x00\xa0\xfc\x8e\xd5\x89\x05\x15\x00\x00\x00\x00\xb1\xf9{@\xfd\xff\xff?\n,\x17@\xfa\xcd\x9a\x99\xfaj\x81@\xfe\xff\xff\x1f\x9e\xa2p@\x00\x10\xc6Aa\x8a\x05\x15\x00\x00\x00\x00\xe03x@\xf9\xff\xff\x1fe9\x1

Opening this in a text editor or with the **open()** function makes most of it look like nonsense!  But with the right program:

In [16]:
dd = df2bin.from_binary_file(filename="hi.bin", formatter=fmt)
dd.head()

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,1514764800000000000,380.047791,5.311336,416.328908,259.994904
1,1514765400000000000,453.769196,5.672167,519.917511,268.641113
2,1514766000000000000,306.376587,5.216037,390.900016,272.564789
3,1514766600000000000,419.645905,5.659674,516.127569,271.258087
4,1514767200000000000,380.650696,5.577941,491.702972,265.674286


Back to normal!  Well, except for the timestamp. 

The main problem with this approach is that it used custom code that isn't tested, isn't available to everyone, and has not been made compatible with a varety of data formats.  We'll look at a few standards out there for storing tabular data in binary formats, but these factors should always be kept in mind.

The advantages of using binary files are many, though:
 - Smaller File Sizes
 - Supports more features (compression, multiple dataset storage, self-description, etc)
 - Quicker read/write times
 - Whole ecosystems of supported software
 

### Pickle

Python's [**pickle**](https://docs.python.org/3/library/pickle.html) format makes it easy to store any Python objects.  The main thing to remember, though, is that they will only read back correctly if the Python version and package versions of the readers are the same as the writer, so pickle files are best used in same-computer situations (saving data between sessions on your own computer) or in strongly managed scenarios.

The pickle module has the same interface as the json module:

| function | read/write | file/string |
| :---:    | :----:     |  :-----:    |
| load()   |  read      | file        |
| dump()   |  write     | file        |
| loads()  |  read      | string      |
| dumps()  |  write     | string      |

#### Pickle module

In [17]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(df, f)

In [18]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
data.head(2)

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,1514764800000000000,380.047791,5.311336,416.328908,259.994904
1,1514765400000000000,453.769196,5.672167,519.917511,268.641113


#### Pandas [to_pickle()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html)

In [19]:
df.to_pickle('data.pickle')

In [20]:
pd.read_pickle('data.pickle').head(2)

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,1514764800000000000,380.047791,5.311336,416.328908,259.994904
1,1514765400000000000,453.769196,5.672167,519.917511,268.641113


### Matlab .mat files

Matlab also has a specific format, although it has changed in recent years.  The **scipy.io** package has two functions for reading .mat files: [**loadmat()**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html) and [**savemat()**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.savemat.html).

In [21]:
from scipy.io import loadmat, savemat
savemat('data.mat', {'data': df.to_records()})

In [22]:
loadmat('data.mat')['data'][0]

array([(array([[0]], dtype=int64), array([[1514764800000000000]], dtype=int64), array([[380.04779053]]), array([[5.31133604]]), array([[416.32890782]]), array([[259.99490356]])),
       (array([[1]], dtype=int64), array([[1514765400000000000]], dtype=int64), array([[453.76919556]]), array([[5.67216682]]), array([[519.91751106]]), array([[268.64111328]])),
       (array([[2]], dtype=int64), array([[1514766000000000000]], dtype=int64), array([[306.37658691]]), array([[5.2160368]]), array([[390.90001581]]), array([[272.56478882]])),
       ...,
       (array([[50527]], dtype=int64), array([[1546299000000000000]], dtype=int64), array([[2201.10693359]]), array([[8.43535805]]), array([[1788.28475526]]), array([[84.74250031]])),
       (array([[50528]], dtype=int64), array([[1546299600000000000]], dtype=int64), array([[2515.6940918]]), array([[9.42136574]]), array([[2418.38250336]]), array([[84.2979126]])),
       (array([[50529]], dtype=int64), array([[1546300200000000000]], dtype=int64), ar

It's not great, I know. 

### HDF5 Files

HDF5 ("Hierarchical Data Format 5") is a file format that has become quite popular.  It can store a lot of data in a single file (Terabytes, even!), has compression features, and can store many datasets.  It has a filesystem-like organization system inside it, which means you kind of store datasets in the format in their own "files" inside the file.  

#### pandas [read_hdf()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_hdf.html) and [to_hdf()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html) functions

To get data insto an hdf5 file, you need to specify both the filename and the **key**/**group** of the dataset--that's the "file in a file".  If you don't give a path, it will put the key in the root group (the "root folder").

In [23]:
df.to_hdf('data.h5', key='/data', )

In [24]:
pd.read_hdf('data.h5', key='/data').head()

,Timestamp,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (deg)
0,1514764800000000000,380.047791,5.311336,416.328908,259.994904
1,1514765400000000000,453.769196,5.672167,519.917511,268.641113
2,1514766000000000000,306.376587,5.216037,390.900016,272.564789
3,1514766600000000000,419.645905,5.659674,516.127569,271.258087
4,1514767200000000000,380.650696,5.577941,491.702972,265.674286


One additional thing to know is that Pandas uses different libraries to save the file depending on the "format" you specify:

| format | package | description |
| :---:  | :-----: | :--------- |
| "table" | pytables | compression and out-of-core querying, but only pytables compatible |
| "fixed" | h5py  | somewhat simpler data format |

#### [h5py](http://docs.h5py.org/en/stable/)

h5py is a low-level file writer and parser for hdf5.  Though it's low-level, it's quite easy to use, and I generally prefer to use it to have more control over how the HDF5 files are formatted.

In [ ]:
import h5py
with h5py.File('data.h5', 'w') as f:
    f.create_dataset("data", data=df.to_records())

In [ ]:
with h5py.File('data.h5') as f:
    data = f['data'][:]